In [146]:
# requests will allow us to make a web request
import requests
# BeautifulSoup will allow us to easily parse the website's HTML code
from bs4 import BeautifulSoup
import pandas as pd

In [147]:
website_url=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [148]:
soup = BeautifulSoup(website_url,'lxml')

In [149]:
My_tables = soup.find_all('table',{'class':'wikitable sortable'})

In [150]:
for table in My_tables:
    ths=table.find_all('th')
    headings=[th.text.strip() for th in ths]

headings 
df=pd.DataFrame(columns=headings)

In [151]:
for tr in table.find_all('tr'):
    tds=tr.find_all('td')
    if not tds:
        continue
    Postcode, Borough, Neighborhood = [td.text.strip() for td in tds]
    df=df.append({'Postcode':Postcode,'Borough':Borough,'Neighbourhood':Neighborhood},ignore_index=True)

In [152]:
#Ignore cells with a borough that is Not assigned.
df=df[df['Borough']!="Not assigned"]

In [153]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
temp=df[df['Neighbourhood']=="Not assigned"]['Borough']
df.loc[df['Neighbourhood']=="Not assigned",'Neighbourhood']=temp

In [154]:
#combine neighbourhoods of same postcode and borough
df['Neighbourhood'] = df.groupby(['Postcode'])['Neighbourhood'].transform(lambda x: ','.join(x))
df=df[['Postcode','Borough','Neighbourhood']].drop_duplicates()

In [155]:
df.shape

(103, 3)